### Cleaning/Processing Steps
The data will be demeaned similar to Zali et al., using a more conservative downsample rate of 20 Hz, given volcanic tremor is typically between 1-9 Hz.

*Read in mseed files <br>
*Check all are the same length <br>
*Demean and detrend <br>
*Anti-alias filter and downsample data to 8 Hz from 100 Hz

In [1]:
import warnings

# Suppress the specific warning from obspy
warnings.filterwarnings(
    "ignore",
    message="The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.*",
    category=UserWarning,
    module="obspy.io.mseed.core"
)

In [2]:
import os
from obspy import read
import glob
import numpy as np

# Define the folder paths
input_folder = os.getcwd() + '/data/raw'
processed_folder = os.getcwd() + '/data/processed'
# number of files in data/raw

files_to_process = len(glob.glob(f'{input_folder}/*.mseed'))
count = 1

# Display folder paths for confirmation
# display(input_folder)
# display(processed_folder)

os.makedirs(processed_folder, exist_ok=True)  # Create output directory if it doesn't exist

# Target sampling rate after downsampling
target_sampling_rate = 8  # Hz

# Initialize a list to store trace lengths
lengths = []

# Process each mseed file in the folder
for file_path in glob.glob(f"{input_folder}/*.mseed"):
    try:
        # Read the file
        st = read(file_path)
        st_check = st.copy()
        
        # Process each trace
        for tr in st:
            # Preprocessing steps
            tr.detrend("demean")  # Remove mean
            tr.detrend("linear")  # Remove linear trend
            tr.filter("lowpass", freq=target_sampling_rate / 2)  # Anti-aliasing filter
            tr.resample(target_sampling_rate)  # Downsample to 8 Hz
            
            tr.normalize()  # Normalize the trace to between -1 and 1
            
            # Record trace length
            lengths.append(len(tr.data))
        
        # Save the processed data
        output_file = os.path.join(processed_folder, os.path.basename(file_path))
        st.write(output_file, format="MSEED")
        print(f'Processed file {count}/{files_to_process}: {os.path.basename(file_path)}')
    
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        continue

    count += 1

# Convert lengths to a NumPy array
lengths = np.array(lengths)
print(f"Lengths of processed traces: {lengths}")


Processed file 1/101: 47_20210425_9fnuph.mseed
Processed file 2/101: 61_20210509_9fnuph.mseed
Processed file 3/101: 24_20210402_9fnuph.mseed
Processed file 4/101: 70_20210518_9fnuph.mseed
Processed file 5/101: 02_20210313_9fnuph.mseed
Processed file 6/101: 37_20210415_9fnuph.mseed
Processed file 7/101: 48_20210426_9fnuph.mseed
Processed file 8/101: 79_20210527_9fnuph.mseed
Processed file 9/101: 83_20210531_9fnuph.mseed
Processed file 10/101: 56_20210504_9fnuph.mseed
Processed file 11/101: 104_20210621_9fnuph.mseed
Processed file 12/101: 22_20210331_9fnuph.mseed
Processed file 13/101: 103_20210620_9fnuph.mseed
Processed file 14/101: 88_20210605_9fnuph.mseed
Processed file 15/101: 36_20210414_9fnuph.mseed
Processed file 16/101: 44_20210422_9fnuph.mseed
Processed file 17/101: 95_20210612_9fnuph.mseed
Processed file 18/101: 14_20210323_9fnuph.mseed
Processed file 19/101: 17_20210326_9fnuph.mseed
Processed file 20/101: 102_20210619_9fnuph.mseed
Processed file 21/101: 08_20210317_9fnuph.msee